#Studi Kasus: Menerapkan K-NN untuk sistem rekomendasi produk

In [1]:
%pip install pandas numpy scikit-learn

  Using cached pandas-2.2.3-cp313-cp313-macosx_11_0_arm64.whl.metadata (89 kB)
  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.3-cp313-cp313-macosx_11_0_arm64.whl (11.3 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 2.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 3.5 MB/s eta 0:00:00a 0:00:01
Using cached pytz-2024.2-py2.py3-none-any.whl (508 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.1/23.1 MB 2.2 MB/s eta 0:00:0000:0100:01m
Using cached tzdata-2024.2-py2.py3-none-any.whl (346 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import Libraries
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler


In [4]:
# Dummy Dataset
# Produk dan fitur yang relevan
data = {
    'Product_ID': ['P1', 'P2', 'P3', 'P4', 'P5'],
    'Feature1': [3, 1, 4, 2, 5],
    'Feature2': [5, 1, 3, 4, 2],
    'Feature3': [4, 2, 5, 1, 3],
}

In [5]:
# Preferensi pelanggan (rating dummy)
customer_preferences = {
    'Customer_ID': ['C1', 'C2', 'C3', 'C4'],
    'P1': [5, 3, 4, 2],
    'P2': [3, 4, 2, 5],
    'P3': [4, 5, 3, 1],
    'P4': [1, 2, 5, 4],
    'P5': [2, 1, 4, 5],
}

In [6]:
#Convert to DataFrames
product_df = pd.DataFrame(data)
customer_df = pd.DataFrame(customer_preferences).set_index('Customer_ID')

In [7]:
# Preprocessing
# Scale product features for distance calculations
scaler = StandardScaler()
scaled_features = scaler.fit_transform(product_df.iloc[:, 1:])


In [8]:
# Implementing K-NN on Customers
# Transpose customer ratings for similarity matching
customer_ratings_matrix = customer_df.T

In [9]:
# Train KNN model for customers
knn_model = NearestNeighbors(n_neighbors=2, metric='euclidean')
knn_model.fit(customer_ratings_matrix)

NearestNeighbors(metric='euclidean', n_neighbors=2)

In [23]:
# Transpose customer ratings for similarity matching (training input)
customer_ratings_matrix = customer_df.T

# Train KNN model for customers
knn_model = NearestNeighbors(n_neighbors=2, metric='euclidean')
knn_model.fit(customer_ratings_matrix.values.T)  # Use the transposed ratings as features

# Updated recommend_products function
def recommend_products(customer_id, customer_df, knn_model, n_recommendations=2):
    # Ensure we query using the transposed matrix (as the model expects this format)
    customer_ratings_matrix = customer_df.T
    
    # Find the index of the customer in the matrix
    customer_idx = list(customer_df.index).index(customer_id)
    customer_ratings = customer_ratings_matrix.values.T[customer_idx].reshape(1, -1)
    
    # Find nearest neighbors (most similar customers)
    distances, indices = knn_model.kneighbors(customer_ratings, n_neighbors=n_recommendations+1)  # +1 to exclude self
    
    # Get indices of similar customers excluding the customer itself
    similar_customer_indices = indices[0][1:]  # Skip the first, which is the customer itself
    similar_customers = [list(customer_df.index)[idx] for idx in similar_customer_indices]
    
    # Collect products rated highly by similar customers
    recommended_products = set()
    for similar_customer in similar_customers:
        liked_products = customer_df.loc[similar_customer][customer_df.loc[similar_customer] >= 4].index.tolist()
        recommended_products.update(liked_products)
    
    # Exclude products already highly rated by the customer
    already_liked = customer_df.loc[customer_id][customer_df.loc[customer_id] >= 4].index.tolist()
    recommended_products.difference_update(already_liked)
    
    return list(recommended_products)




In [24]:
# Test the corrected recommendation function
customer_id = 'C1'
recommendations = recommend_products(customer_id, customer_df, knn_model)

print(f"\nRecommended products for Customer {customer_id}: {recommendations}")


Recommended products for Customer C1: ['P5', 'P2', 'P4']
